In [1]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service as FirefoxService
import selenium.webdriver.support.expected_conditions as ec
from webdriver_manager.firefox import GeckoDriverManager

In [2]:
def format_date(date):
    """
    Функция принимает на вход дату в формате день/месяц/год и возвращает в формате год-месяц-день
    
    Параметры:
        date: string — дата
    """
    
    day, month, year = date.split("/")
    
    return "{}-{}-{}".format(year, month, day)

In [3]:
def get_rsd_to_usd(date_start, date_end, web_driver, save=False, filename=None):
    """
    Функция принимает на вход начало и конец периода, за которые нужно получить данные о курсе обмена
    сербского динара на американский доллар.
    Возвращает DataFrame со столбцами [Date, Value]. Если указана опция save, то сохраняет созданный 
    DataFrame с указанным именем в формате .xlsx
    
    Параметры:
        date_start: string — начало периода,
        date_end: string — конец периода,
        save: boolean — сохранять данные в excel-файл или нет, по умолчанию save=False,
        filename: string — если save=True, то является обязательным, название excel-файла,
        web_driver: string — путь к драйверу Firefox
    """
    
    if save and not filename:
        raise ValueError("Filename should be specified when save option is True")
    
    # опция headless позволяет не показывать браузер во время работы функции
    options = Options()
    options.headless = True
    
    # открываем браузер с указанной опцией
    driver = webdriver.Firefox(service=FirefoxService(web_driver), options=options)
    driver.get("https://nbs.rs/kursnaListaModul/inputPeriod.faces")
    
    # находим нужные нам элементы
    date_start_element = driver.find_element(By.ID, "inputPeriod:Od")
    date_end_element = driver.find_element(By.ID, "inputPeriod:Do")

    currency_element = driver.find_element(By.ID, "inputPeriod:valutaInner")
    currency_select = Select(currency_element)

    rate_type_element = driver.find_element(By.ID, "inputPeriod:vrstaInner")
    rate_type_select = Select(rate_type_element)

    show_element = driver.find_element(By.ID, "inputPeriod:prikazInner")
    show_select = Select(show_element)

    show_button = driver.find_element(By.ID, "inputPeriod:buttonShow")
    
    # указываем в форме даты, за которые нужно получить данные
    date_start_element.clear()
    date_start_element.send_keys(date_start)

    date_end_element.clear()
    date_end_element.send_keys(date_end)
    
    # в селекторах выбираем валюту, тип курса, вид выдачи данных
    currency_select.select_by_value("840")
    rate_type_select.select_by_value("3")
    show_select.select_by_value("html")
    
    # переходим на страницу с данными
    show_button.click()
    
    # ожидаем, пока не появится таблица с данными, после чего получаем её
    rates = WebDriverWait(driver, 20).until(ec.visibility_of_element_located((By.NAME, "map1")))
    rates_HTML = rates.get_attribute("innerHTML")
    
    # из HTML выделяем сам курс валют
    regexp = re.compile("(?<=\(dd\/mm\)\: ).*?(?= RSD)")
    rates_string = re.findall(regexp, rates_HTML)
    rates = [rate.split(": ")[1] for rate in rates_string]
    
    # генерируем список дат, переводим в нужный нам формат
    date_range = pd.date_range(start=format_date(date_start), end=format_date(date_end)).strftime('%d/%m/%Y')
    
    # формируем DataFrame
    exchange_rates = [(element[0], element[1]) for element in zip(date_range, rates)]
    exchange_rates_df = pd.DataFrame(exchange_rates, columns=['Date', 'RSD'])
    exchange_rates_df['RSD'] = exchange_rates_df['RSD'].astype(float)
    
    if save:
        exchange_rates_df.to_excel(filename, index=False)
        
    return exchange_rates_df

In [4]:
date_start = "10/10/2022"
date_end = "18/10/2022"

In [5]:
web_driver = GeckoDriverManager().install()

[WDM] - Downloading: 19.0kB [00:00, 19.5MB/s]                                                                       
[WDM] - Downloading: 19.0kB [00:00, 4.92MB/s]                                                                       
[WDM] - Downloading: 100%|█████████████████████████████████████████████████████| 1.58M/1.58M [00:01<00:00, 1.46MB/s]


In [6]:
get_rsd_to_usd(date_start, date_end, web_driver=web_driver, save=True, filename="exchange_rates_rsd.xlsx")

,Date,RSD
0,10/10/2022,120.4423
1,11/10/2022,121.1413
2,12/10/2022,120.7371
3,13/10/2022,120.8833
4,14/10/2022,119.8641
5,15/10/2022,119.8641
6,16/10/2022,119.8641
7,17/10/2022,120.3232
8,18/10/2022,118.9927
